# Graph Viewer Demo

## 1. View a CX graph in an interactive viewer

In [18]:
from IPython.display import display
import json

data = ''
with open('example.cx') as f:
    data = f.read()

data_json = json.loads(data)
cx_data = {
    "application/cx": data_json
}

display(cx_data,raw=True)

In [10]:
cyjs_graph = {
  'elements': {
    'nodes': [
      {'data': {'id': 1, 'name': 'test1'}},
      {'data': {'id': 2, 'name': 'test2'}}

    ],
    'edges': [{'data': {'source': 1, 'target': 2}}]
  },
  'data': {
    'name': 'Test1'
  }
}

cy_data = {
    "application/cx": cyjs_graph
}

display(cy_data,raw=True)

In [11]:
import pandas as pd
from pprint import pprint
from collections import defaultdict
from mygene import MyGeneInfo
import requests
import graphviz as gv
mg = MyGeneInfo()


In [12]:
class BioLinkWrapper(object):
    def __init__(self):
        self.endpoint = 'https://api.monarchinitiative.org/api/'
        self.params = {
            'fetch_objects': 'true',
        }

    def get_obj(self, obj_id):
        url = '{0}bioentity/{1}'.format(self.endpoint, obj_id)
        response = requests.get(url)
        return response.json()

    def disease2genes(self, disease_curie):
        params = {}
        url = '{0}bioentity/disease/{1}/genes'.format(self.endpoint, disease_curie)
        response = requests.get(url, params)
        return response.json()

    def disease2phenotypes(self, disease_curie):
        params = {}
        url = '{0}bioentity/disease/{1}/phenotypes'.format(self.endpoint, disease_curie)
        response = requests.get(url, params)
        return response.json()

    def gene(self, gene_curie):
        params = {}
        url = '{0}bioentity/gene/{1}'.format(self.endpoint, gene_curie)
        response = requests.get(url, params)
        return response.json()

    def gene2orthologs(self, gene_curie, orth_taxon_name=None):
        taxon_map = {
            'mouse': 'NCBITaxon:10090',
            'rat': 'NCBITaxon:10116',
            'zebrafish': 'NCBITaxon:7955',
            'fly': 'NCBITaxon:7227',
            'worm': 'NCBITaxon:6239'
        }
        params = {}
        if orth_taxon_name:
            params['homolog_taxon'] = taxon_map[orth_taxon_name]
        url = '{}bioentity/gene/{}/homologs/'.format(self.endpoint, gene_curie)
        response = requests.get(url, params)
        return response.json()

    def gene2phenotypes(self, gene_curie):
        url = '{}bioentity/gene/{}/phenotypes/'.format(self.endpoint, gene_curie)
        response = requests.get(url)
        return response.json()

    def gene2diseases(self, gene_curie):
        url = '{}bioentity/gene/{}/diseases/'.format(self.endpoint, gene_curie)
        response = requests.get(url)
        return response.json()

    def gene_interactions(self, gene_curie):
        url = '{}bioentity/gene/{}/interactions/'.format(self.endpoint, gene_curie)
        response = requests.get(url)
        return response.json()

    def gene2functions(self, gene_curie):
        url = '{}bioentity/gene/{}/function/'.format(self.endpoint, gene_curie)
        response = requests.get(url)
        return response.json()

    def gene2tissue_expression(self, gene_curie):
        url = '{}bioentity/gene/{}/expression/anatomy/'.format(self.endpoint, gene_curie)
        response = requests.get(url)
        return response.json()

    def tissue2gene_expression(self, tissue_curie):
        url = '{}bioentity/anatomy/{}/genes/'.format(self.endpoint, gene_curie)
        response = requests.get(url)
        return response.json()

    def disease_models(self, disease_curie):
        url = '{}/bioentity/disease/{}/models/'.format(self.endpoint, disease_curie)
        response = requests.get(url)
        return response.json()

    def taxon2phenotypes(self, taxon_curie):
        # get phenotypes associated with taxid
        url = "mart/gene/phenotype/{}".format(self.endpoint, taxon_curie)
        response = requests.get(url)
        return response.json()

    def parse_gene_functions(self, curie):
        function_list = list()
        functions = self.gene2functions(gene_curie=curie)
        if 'associations' in functions.keys():
            for assoc in functions['associations']:
                function_list.append(assoc['object']['label'])
        function_set = set(function_list)
        return ", ".join(function_set)

    def get_orthoglog_gene_set(self, gene_set, orth_taxon_name):
        orth_set = []
        for gene in gene_set:
            orth_set.append(self.gene2orthologs(gene_curie=gene, orth_taxon_name=orth_taxon_name))
        return orth_set


    @staticmethod
    def parse_sources(sources):
        return [x.split('/')[-1].rstrip('.ttl') for x in sources]

    @staticmethod
    def parse_association(input_id, input_label, association, invert_subject_object=False):
        hit_id = association['object']['id']
        hit_label = association['object']['label']
        if invert_subject_object:
            hit_id = association['subject']['id']
            hit_label = association['subject']['label']
        parsed_association = {
            'input_id': input_id,
            'input_label': input_label,
            'hit_id': hit_id,
            'hit_label': hit_label,
            'sources': BioLinkWrapper.parse_sources(association['provided_by']),
        }
        return parsed_association

    @staticmethod
    def return_objects(assoc_package):
        return assoc_package['objects']




In [13]:
blw = BioLinkWrapper()

In [14]:
def conv_pref(prefix):
    return prefix.replace(':', '_')

def map_id(in_id):
    return blw.get_obj(obj_id=in_id)['id']

In [16]:
# workflow input is a disease identifier
fanconi_anemia = 'MONDO:0019391'

# Begin Graphviz representation of workflow
wf2_graph = gv.Digraph(name="Workfow II Fanconi Anemia Implementation", format='json')
wf2_graph.node(conv_pref(fanconi_anemia), label='Fanoni Anemia')

wf2_graph.node('Mod1A', label='Mod1A Functional Similarity')
wf2_graph.node('Mod1B', label='Mod1B Phenotype Similarity')
wf2_graph.node('Mod1C', label='Mod1C Coexpression')
wf2_graph.node('Mod1D', label='Mod1D Gene Chemical Interaction')
wf2_graph.node('Mod1E', label='Mod1E Gene Interaction')
wf2_graph.node('Mod3', label='Mod3 Candidate Merging')

In [20]:
fa_gene_associations = blw.disease2genes(disease_curie=fanconi_anemia)
parsed_fa_genes = [blw.parse_association(fanconi_anemia, x) for x in fa_gene_associations['associations']]


TypeError: parse_association() missing 1 required positional argument: 'association'

In [19]:
# load into pandas dataframe


for fa_gene in parsed_fa_genes:
    fa_mg = mg.query(fa_gene['hit_id'].replace('HGNC', 'hgnc'), species='human', entrezonly=True, fields='entrez,HGNC,symbol')
    fa_gene.update({'ncbi': 'NCBIGene:{}'.format(fa_mg['hits'][0]['_id'])})
    
    

TypeError: parse_association() missing 1 required positional argument: 'association'

In [17]:
fa_genes_df = pd.DataFrame(data=parsed_fa_genes)
# group duplicate ids and gather sources
fa_genes_df['sources'] = fa_genes_df['sources'].str.join(', ')

TypeError: parse_association() missing 1 required positional argument: 'association'

In [ ]:
import ndex2.client as nc
import io
import json
from IPython.display import HTML
from time import sleep
anon_ndex=nc.Ndex2("http://public.ndexbio.org")
anon_ndex.update_status()

#the string of below command is match the uuid of the network which you want to draw in ndex page.
response=anon_ndex.get_network_as_cx_stream('54a9a35b-1e5f-11e8-b939-0ac135e8bacf')
print("Received %s characters of CX" % len(response.content))
raw_cx = response.json()

#raw_cx= json.loads(response)

cx_data = {
    "application/cx": raw_cx
}
display(cx_data,raw=True)